In [1]:
from evaluate import *
import os
from sys import argv
import sys
import tensorflow as tf

parent_dir  = "/u/skhan/stem-learning/"

sys.path.insert(0, '{}code/cycle_gan'.format(parent_dir))
from models import unet_generator

idfn        = "107" #argv[1]
model_type  = "sim" #argv[2]
lbl         = "1vacancy"

identifier  = "20220606_MODEL_unet_dist_gen_fft_10_SIM_pristine_gaussian_0.1_EXP_{}".format(idfn)
results_dir = "{}results/{}/results_{}/{}/".format(parent_dir, model_type, identifier, lbl)
save_dir    = "{}evals/".format(results_dir)
gan_cpt_dir = "{}cycle_gan_results/checkpoints/checkpoint_{}".format(parent_dir, identifier)

In [2]:
if model_type == "sim":
    generator_exp = unet_generator(1, 1, "instancenorm")
    generator_sim = unet_generator(1, 1, "instancenorm")

    def cycle(x):
        return generator_sim(generator_exp(x))
    
    try:
        ckpt = tf.train.Checkpoint(generator_exp=generator_exp, generator_sim=generator_sim)
        ckpt_manager = tf.train.CheckpointManager(ckpt, gan_cpt_dir, max_to_keep=3)
        if ckpt_manager.latest_checkpoint:
            cpath = ckpt_manager.latest_checkpoint
            ckpt.restore(cpath)
    except Exception as e:
        print("loading checkpoint failed")
        print(e)

2022-06-27 15:59:14.771003: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-27 15:59:15.249461: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38270 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0


In [3]:
def cycle_image(cycle, input_file, save_dir, fname, num_channels=1): 
    print("GANNING {}".format(input_file))
    generate_image(cycle, input_file, (256,256), stride=(256,256), avg=False, \
        plot=False, save_data=True, save_dir=save_dir, fname=fname)

In [4]:
def evaluate_image(model_type, data_dir, results_dir, lbl, save_dir, save_fn_prefix):
    
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir, exist_ok=True)

    model_fn         = results_dir + "model.json"                                              
    model_weights_fn = results_dir + "weights.h5"
    input_file = data_dir + "input.tif" 
    if model_type == "sim":
        cycle_image(cycle, input_file, save_dir, "{}_cycled.tif".format(save_fn_prefix))
        input_file = "{}{}_cycled.tif".format(save_dir, save_fn_prefix)

    l_shape = (256, 256)
    stride  = (64,64)
    plot = False
    fname = "evaluated_labels_{}.tiff".format(save_fn_prefix)

    print("EVALUATING {}".format(input_file))
    prediction = evaluate(model_fn, model_weights_fn, input_file, l_shape, stride,
                        avg=True, plot=plot, save_data=True, save_dir=save_dir, fname=fname)

    label_file_list = [data_dir + "label_{}.tif".format(lbl)]
    tol    = 0.5
    nconvs = 2
    r      = 9
    prefix = save_fn_prefix
    TP, FP, FN, TN, recall, precision, F1, bal_acc = calc_accuracy(prediction, label_file_list, tol=tol, bdy=32,
                                                                nconvs=nconvs, r=r, TN=0, plot=plot, 
                                                                save_data=True, save_dir=save_dir, 
                                                                prefix=prefix, verbose=False)
    return TP, FP, FN

In [5]:
if not os.path.isdir(save_dir):
        os.makedirs(save_dir, exist_ok=True)
        
label_csv = "{}label_row.csv".format(save_dir)
def write_to_file(vals): 
    with open(label_csv, "a") as f:
        f.write(vals)

In [6]:
if os.path.exists(label_csv):
    os.remove(label_csv)

In [7]:
write_to_file("{}_{}".format(idfn, model_type))

In [8]:
for (d, pic) in [(211,1528), (211, 1922), (211, 2124), (107, 1750), (107, 1840), (107, 1847)]:
    data_dir = parent_dir + "data/WSe/test_experiment/{}_day/RR_{}/".format(d, pic)
    TP, FP, FN = evaluate_image(model_type, data_dir, results_dir, lbl, save_dir, "{}_{}".format(d, pic))
    write_to_file(",{},{},{}".format(TP,FP,FN))

GANNING /u/skhan/stem-learning/data/WSe/test_experiment/211_day/RR_1528/input.tif
processing data
loading model
strides: (256, 256)
predicting data


2022-06-27 15:59:17.371841: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2022-06-27 15:59:18.049022: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-06-27 15:59:18.049572: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-06-27 15:59:18.049589: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-06-27 15:59:18.049888: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-06-27 15:59:18.049937: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


stitching data
saving data
processing data
loading model
strides: (64, 64)
predicting data


2022-06-27 15:59:19.311875: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


stitching data
saving data
GANNING /u/skhan/stem-learning/data/WSe/test_experiment/211_day/RR_1922/input.tif
processing data
loading model
strides: (256, 256)
predicting data
stitching data
saving data
processing data
loading model
strides: (64, 64)
predicting data
stitching data
saving data
GANNING /u/skhan/stem-learning/data/WSe/test_experiment/211_day/RR_2124/input.tif
processing data
loading model
strides: (256, 256)
predicting data
stitching data
saving data
processing data
loading model
strides: (64, 64)
predicting data
stitching data
saving data
GANNING /u/skhan/stem-learning/data/WSe/test_experiment/107_day/RR_1750/input.tif
processing data
loading model
strides: (256, 256)
predicting data
stitching data
saving data
processing data
loading model
strides: (64, 64)
predicting data
stitching data
saving data
GANNING /u/skhan/stem-learning/data/WSe/test_experiment/107_day/RR_1840/input.tif
processing data
loading model
strides: (256, 256)
predicting data
stitching data
saving data
